In [26]:
import pandas as pd
import numpy as np
import re
from pandas.io.json import json_normalize

csv_filename = 'product_details_laptop.csv'
csv_bought_together = 'items_bought_together_laptop.csv'
csv_customer_reviews = 'links_to_customer_reviews_laptop.csv'
txt_customer_reviews = 'links_to_customer_reviews_laptop.txt'

In [80]:
# df = pd.read_csv('product_details_laptop.csv', index_col=0)
# df.dtypes


In [27]:
df = pd.read_json('laptop_product_output.jsonl')
df.head()

,name,product_summary,product_tech_spec,product_addl_info,seller,seller_link,freq_bought,freq_bought_link,link_to_all_reviews
0,CHUWI Herobook Pro 14.1 inch Windows 10 Intel ...,"[{'info': 'Brand', 'value': 'CHUWI'}, {'info':...","[{'info': 'Standing screen display size', 'val...","[{'info': 'ASIN', 'value': 'B085XRL3P1'}, {'in...",CHUWI,https://www.amazon.com/gp/help/seller/at-a-gla...,VicTsing MM057 2.4G Wireless Portable Mobile M...,https://www.amazon.com/VicTsing-Wireless-Porta...,/CHUWI-Herobook-Pro-Notebook-Lightweight/produ...
1,CHUWI HeroBook Pro 14.1 inch Windows 10 Laptop...,"[{'info': 'Brand', 'value': 'CHUWI'}, {'info':...","[{'info': 'Standing screen display size', 'val...","[{'info': 'ASIN', 'value': 'B08316YSKH'}, {'in...",ChuwiDirect,https://www.amazon.com/gp/help/seller/at-a-gla...,VicTsing MM057 2.4G Wireless Portable Mobile M...,https://www.amazon.com/VicTsing-Wireless-Porta...,/CHUWI-HeroBook-Pro-Notebook-Lightweight/produ...
2,"ASUS Laptop L210 Ultra Thin Laptop, 11.6” HD D...","[{'info': 'Brand', 'value': 'ASUS'}, {'info': ...","[{'info': 'Standing screen display size', 'val...","[{'info': 'ASIN', 'value': 'B081V6W99V'}, {'in...",AJay Sales,https://www.amazon.com/gp/help/seller/at-a-gla...,None,None,/ASUS-Processor-NumberPad-Microsoft-L210MA-DB0...
3,"HP Stream 14-inch Laptop, Intel Celeron N4000,...","[{'info': 'Brand', 'value': 'HP'}, {'info': 'S...","[{'info': 'Standing screen display size', 'val...","[{'info': 'ASIN', 'value': 'B084SMMH96'}, {'in...",Kailone Tech,https://www.amazon.com/gp/help/seller/at-a-gla...,VicTsing 2.4G Slim Wireless Mouse with Nano Re...,https://www.amazon.com/VicTsing-Wireless-Recei...,/HP-14-cb186nr-Brilliant-9MV74UA-ABA/product-r...
4,"Lenovo IdeaPad 3 15"" Laptop, 15.6"" HD (1366 x ...","[{'info': 'Brand', 'value': 'Lenovo'}, {'info'...","[{'info': 'Standing screen display size', 'val...","[{'info': 'ASIN', 'value': 'B0872K4XCJ'}, {'in...",None,None,VicTsing MM057 2.4G Wireless Portable Mobile M...,/VicTsing-Wireless-Portable-Receiver-Adjustabl...,/Lenovo-Processor-Graphics-81W10094US-Business...


In [28]:
df.name = df.name.str.replace('"','inch')

In [29]:
df = df.dropna(axis=0, how='all')

In [30]:
df.link_to_all_reviews = 'https://www.amazon.com/' + df.link_to_all_reviews

In [31]:
df.reset_index(drop=True, inplace = True)

In [32]:
# Extraction of all specifications in nested JSON-dicts to be used for building additional columns 
  
new_columns = []
for i in range(len(df)):
    if df.product_summary[i] is not None:
        product_sum = df.product_summary[i]
        x = len(product_sum)  
        for j in range(x):
            new_columns.append(product_sum[j]['info'])
    if df.product_tech_spec[i] is not None:
        product_tech = df.product_tech_spec[i]
        y = len(product_tech)  
        for j in range(y):
            new_columns.append(product_tech[j]['info'])
    if df.product_addl_info[i] is not None:
        product_add = df.product_addl_info[i]
        z = len(product_add)  
        for j in range(z):
            new_columns.append(product_add[j]['info'])

In [33]:
for i in range(len(new_columns)):
    if new_columns[i] is not None:
        new_columns[i] = new_columns[i].lower()
new_columns = set(new_columns) 

In [34]:
new_columns = list(new_columns)

In [35]:
try:
    new_columns.remove(None)
except:
    pass

In [36]:
for i in new_columns:
    df[i] = i
    df[i] = ''

In [37]:
for i in range(len(df)):
    if df.product_summary[i] is not None:
        psc = []
        product_sum = df.product_summary[i]
        product_sum = pd.DataFrame(product_sum)
        product_sum = product_sum.transpose()
        psc_len = len(product_sum.columns)  # collecting number of columns in product_sum / items in df.product_summary
        for j in range(psc_len):            # getting item / column names
            psc.append(product_sum[j][0])
        for k in range(len(psc)):           # setting item / column names in lower cases for transfer of values into correct df columns
            if psc[k] is not None:
                psc[k] = psc[k].lower()
        product_sum.columns = product_sum.iloc[0] # Turns row 1 into column names
        product_sum = product_sum.drop('info') # Drops row 1
        product_sum.columns = psc
        for l in psc: 
            try:
                df[l][i] = product_sum[l]['value']
            except:
                continue
    if df.product_tech_spec[i] is not None:
        ptc = []
        product_tec = df.product_tech_spec[i]
        product_tec = pd.DataFrame(product_tec)
        product_tec = product_tec.transpose()
        ptc_len = len(product_tec.columns)  # collecting number of columns in product_tec / items in df.product_tech_spec
        for j in range(ptc_len):            # getting item / column names
            ptc.append(product_tec[j][0])
        for k in range(len(ptc)):           # setting item / column names in lower cases for transfer of values into correct df columns
            if ptc[k] is not None:
                ptc[k] = ptc[k].lower()
        product_tec.columns = product_tec.iloc[0] # Turns row 1 into column names
        product_tec = product_tec.drop('info') # Drops row 1
        product_tec.columns = ptc
        for l in ptc:   ## What does this loop through??? I need the column name, but I have to stay within one row!!!
            try:
                df[l][i] = product_tec[l]['value']
            except:
                continue
    if df.product_addl_info[i] is not None:
        pac = []
        product_add = df.product_addl_info[i]
        product_add = pd.DataFrame(product_add)
        product_add = product_add.transpose()
        pac_len = len(product_add.columns)  # collecting number of columns in product_tec / items in df.product_tech_spec
        for j in range(pac_len):            # getting item / column names
            pac.append(product_add[j][0])
        for k in range(len(pac)):           # setting item / column names in lower cases for transfer of values into correct df columns
            if pac[k] is not None:
                pac[k] = pac[k].lower()
        product_add.columns = product_add.iloc[0] # Turns row 1 into column names
        product_add = product_add.drop('info') # Drops row 1
        product_add.columns = pac
        for l in pac:   ## What does this loop through??? I need the column name, but I have to stay within one row!!!
            try:
                df[l][i] = product_add[l]['value']
            except:
                continue

In [38]:
df.rename(columns = {'asin' : 'ASIN'}, inplace = True)

In [53]:
df['customer reviews'] = df['customer reviews'].str.split('{\ntext-decoration: none;\n} ').str[1]
df['customer reviews'] = df['customer reviews'].str.split(' out of 5 stars').str[0]
df['customer reviews'] = df['customer reviews'].str.split(' von 5 Sternen').str[0]

# df['ASIN'] = df.url.str.split('/dp/').str[-1]
# df['ASIN'] = df.ASIN.str.split('%2Fdp%2F').str[-1]
# df['ASIN'] = df.ASIN.str.split('/').str[0]
# df['ASIN'] = df.ASIN.str.split('%2F').str[0]

In [54]:
pd.options.display.max_colwidth = 500
df['customer reviews']

0         4.0 out of 5 stars 419 ratings P.when('A', 'ready').execute(function(A) {\nA.declarative('acrLink-click-metrics', 'click', { "allowLinkDefault" : true }, function(event){\nif(window.ue) {\nue.count("acrLinkClickCount", (ue.count("acrLinkClickCount") || 0) + 1);\n}\n});\n}); P.when('A', 'cf').execute(function(A) {\nA.declarative('acrStarsLink-click-metrics', 'click', { "allowLinkDefault" : true },  function(event){\nif(window.ue) {\nue.count("acrStarsLinkWithPopoverClickCount", (ue.count("ac...
1         4.0 out of 5 stars 619 ratings P.when('A', 'ready').execute(function(A) {\nA.declarative('acrLink-click-metrics', 'click', { "allowLinkDefault" : true }, function(event){\nif(window.ue) {\nue.count("acrLinkClickCount", (ue.count("acrLinkClickCount") || 0) + 1);\n}\n});\n}); P.when('A', 'cf').execute(function(A) {\nA.declarative('acrStarsLink-click-metrics', 'click', { "allowLinkDefault" : true },  function(event){\nif(window.ue) {\nue.count("acrStarsLinkWithPopoverClickCount",

In [56]:
df.to_csv(csv_filename)

In [55]:
zaehler = 0
for i in range(len(df)):
    if df['customer reviews'][i] is not None:
        try:
            res = [re.findall(r'(\d+),(\d+) von 5 Sternen', df['customer reviews'][i])[0] ]
            a = res[0][0]
            b = res[0][1]
            c = float(a+'.'+b)
            df['customer reviews'][i] = c
            zaehler += 1
        except:
            pass

In [15]:
for i in range(len(df)):
    if df['customer reviews'][i] is not None:
        try:
            res = [re.findall(r'(\d+).(\d+) out of 5 stars', df['customer reviews'][i])[0] ]
            a = res[0][0]
            b = res[0][1]
            c = float(a+'.'+b)
            df['customer reviews'][i] = c
        except:
            pass
        try:
            res = [re.findall(r'(\d+),(\d+) von 5 Sternen', df['customer reviews'][i])[0] ]
            a = res[0][0]
            b = res[0][1]
            c = float(a+'.'+b)
            df['customer reviews'][i] = c
        except:
            pass
    else:
        df['customer reviews'][i] = 0
df.rename(columns = {'customer reviews' : 'rating'}, inplace = True)

In [16]:
df = df.drop(['product_summary', 'product_tech_spec', 'product_addl_info'], axis = 1)

In [17]:
for i in range(len(df)):
    if df.name[i] is None:
        df = df.drop([i], axis = 0)

In [18]:
# Creating dataframe with links to all customer reviews for saving in additional CSV-file

df_laptop_customer_reviews_link = df.link_to_all_reviews
df_laptop_customer_reviews_link = df_laptop_customer_reviews_link.dropna()
df_laptop_customer_reviews_link.reset_index(drop=True, inplace = True)

In [19]:
df_laptop_bought_together = df[['ASIN', 'name', 'freq_bought', 'freq_bought_link']]

In [20]:
df.to_csv(csv_filename)
#df.to_csv(csv_filename, sep=';')
df_laptop_customer_reviews_link.to_csv(csv_customer_reviews) # actually not necessary anymore since there's being a txt-file created
df_laptop_bought_together.to_csv(csv_bought_together)

In [21]:
f = open(txt_customer_reviews, 'w+')

for i in range(len(df_laptop_customer_reviews_link)):
    f.write(df_laptop_customer_reviews_link[i])
    f.write('\n')

f.close()

In [22]:
df2 = pd.read_csv(csv_filename)
df2

,Unnamed: 0,name,seller,seller_link,freq_bought,freq_bought_link,link_to_all_reviews,anzahl der leuchtmittel,benötigt batterien,best sellers rank,...,modell/serie,speicher-art,is discontinued by manufacturer,produktabmessungen,hardware plattform,modellnummer,internationaler versand,item model number,manufacturer,stilrichtung
0,0,CHUWI Herobook Pro 14.1 inch Windows 10 Intel ...,CHUWI,https://www.amazon.com/gp/help/seller/at-a-gla...,VicTsing MM057 2.4G Wireless Portable Mobile M...,https://www.amazon.com/VicTsing-Wireless-Porta...,https://www.amazon.com//CHUWI-Herobook-Pro-Not...,NaN,NaN,"#4,405 in Computers & Accessories ( See Top 10...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Herobook Pro,NaN,NaN
1,1,CHUWI HeroBook Pro 14.1 inch Windows 10 Laptop...,ChuwiDirect,https://www.amazon.com/gp/help/seller/at-a-gla...,VicTsing MM057 2.4G Wireless Portable Mobile M...,https://www.amazon.com/VicTsing-Wireless-Porta...,https://www.amazon.com//CHUWI-HeroBook-Pro-Not...,NaN,NaN,"#1,136 in Computers & Accessories ( See Top 10...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Herobook Pro,NaN,NaN
2,2,"ASUS Laptop L210 Ultra Thin Laptop, 11.6” HD D...",AJay Sales,https://www.amazon.com/gp/help/seller/at-a-gla...,NaN,NaN,https://www.amazon.com//ASUS-Processor-NumberP...,NaN,NaN,#13 in Computers & Accessories ( See Top 100 i...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L210MA-DB01,NaN,NaN
3,3,"HP Stream 14-inch Laptop, Intel Celeron N4000,...",Kailone Tech,https://www.amazon.com/gp/help/seller/at-a-gla...,VicTsing 2.4G Slim Wireless Mouse with Nano Re...,https://www.amazon.com/VicTsing-Wireless-Recei...,https://www.amazon.com//HP-14-cb186nr-Brillian...,NaN,NaN,#992 in Computers & Accessories ( See Top 100 ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9MV74UA#ABA,NaN,NaN
4,4,"Lenovo IdeaPad 3 15inch Laptop, 15.6inch HD (1...",NaN,NaN,VicTsing MM057 2.4G Wireless Portable Mobile M...,/VicTsing-Wireless-Portable-Receiver-Adjustabl...,https://www.amazon.com//Lenovo-Processor-Graph...,NaN,NaN,#205 in Computers & Accessories ( See Top 100 ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81W10094US,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11563,11574,Schreibtisch Fan Tragbare USB-Mini Air Cooling...,Deals by Dennis Jr,https://www.amazon.com/-/de/gp/help/seller/at-...,NaN,NaN,https://www.amazon.com//-/de/Portable-Cooling-...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,atb-OB826,NaN,NaN,NaN,NaN
11564,11575,"[3-Pack] Webcam Cover, Camera Cover Slide, Lap...",Rukina Int,https://www.amazon.com/-/de/gp/help/seller/at-...,NaN,NaN,https://www.amazon.com//-/de/3-Pack-Laptops-Co...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11565,11576,"Benfei DisplayPort auf VGA, Adapter Stecker au...",BenfeiDirect,https://www.amazon.com/-/de/gp/help/seller/at-...,"Benfei DisplayPort auf VGA-Adapter, 90 cm lang...",https://www.amazon.com/-/de/dp/B082DH5K3L/ref=...,https://www.amazon.com//-/de/DisplayPort-Benfe...,NaN,NaN,NaN,...,NaN,NaN,NaN,0.39 x 0.39 x 0.39 cm,NaN,000227grey,NaN,NaN,NaN,NaN
11566,11577,Used Like New Ultrabook Latitude E7240 Busines...,NaN,NaN,NaN,NaN,https://www.amazon.com//Ultrabook-Latitude-Bus...,NaN,NaN,"#3,381 in Computers & Accessories ( See Top 10...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7420,NaN,NaN


In [101]:
df2[df2.rating == 0]

,Unnamed: 0,name,seller,seller_link,freq_bought,freq_bought_link,link_to_all_reviews,brand name,color,rating,...,color name,ASIN,manufacturer part number,connectivity technologies,assembled width,special features,domestic shipping,os,assembled length,language
